In [1]:
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, classification_report
import glob
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.mixture import GaussianMixture
from keras.models import Sequential
from keras.layers import Dense, Flatten
import matplotlib.pyplot as plt
from scipy.stats import mode


# 数据预处理
def load_images(image_paths, target_size=(256, 256)):
    images = []
    labels = []

    for image_path in image_paths:
        image = Image.open(image_path).convert('RGB').resize(target_size)
        image = np.array(image) / 255.0
        label = image_path.split('/')[-2]
        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)


# 内建的ANN模型
class ANNModel:
    def __init__(self, input_shape, num_classes):
        self.model = Sequential([
            Flatten(input_shape=input_shape),
            Dense(128, activation='relu'),
            Dense(128, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])
        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    def train(self, X_train, y_train):
        return self.model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    def predict(self, X):
        return self.model.predict(X)


# 无监督模型 (GMM)
class UnsupervisedGMM:
    def __init__(self, n_components):
        self.model = GaussianMixture(n_components=n_components)

    def fit(self, X):
        self.model.fit(X)

    def predict(self, X):
        return self.model.predict(X)


# Ensemble模型
class EnsembleModel:
    def __init__(self, supervised_model, unsupervised_model):
        self.supervised_model = supervised_model
        self.unsupervised_model = unsupervised_model

    def fit(self, X, y):
        history = self.supervised_model.train(X, y)
        features = self.supervised_model.predict(X)
        self.unsupervised_model.fit(features)
        return history

    def predict(self, X):
        features = self.supervised_model.predict(X)
        return self.unsupervised_model.predict(features)


# 可视化训练历史
def plot_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.legend()
    plt.show()


# 重新映射聚类标签以匹配真实标签
def remap_labels(true_labels, predicted_labels):
    unique_true_labels = np.unique(true_labels)
    mapped_labels = np.zeros_like(predicted_labels)
    for true_label in unique_true_labels:
        mask = (true_labels == true_label)
        # Find the cluster label that most frequently occurs with this true label
        most_common = mode(predicted_labels[mask])[0][0]
        mapped_labels[predicted_labels == most_common] = true_label
    return mapped_labels


# 加载数据和训练模型
image_paths = glob.glob('Car_Logo_Dataset/**/*.png', recursive=True)
X, y = load_images(image_paths)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
# 构建模型
ann_model = ANNModel(X_train[0].shape, len(np.unique(y_train)))
gmm_model = UnsupervisedGMM(n_components=len(np.unique(y_train)))
ensemble_model = EnsembleModel(ann_model, gmm_model)

# 训练模型
history = ensemble_model.fit(X_train, y_train)

# 可视化损失和准确率
plot_history(history)

# 进行预测
features_test = ann_model.predict(X_test)  # 先通过ANN获得特征

/var/folders/fm/_6wfn0r96gjcpy7779d0j2fc0000gn/T/ipykernel_42386/22398637.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X)


AttributeError: 'NoneType' object has no attribute 'keras'

In [ ]:
features_test.shape

In [ ]:
predicted_clusters = gmm_model.predict(features_test)  # 用GMM进行聚类预测
mapped_predictions = remap_labels(y_test, predicted_clusters)  # 重新映射聚类标签

# 计算混淆矩阵
conf_matrix = confusion_matrix(y_test, mapped_predictions)
print('Confusion Matrix:')
print(conf_matrix)

# 绘制混淆矩阵
plt.figure(figsize=(12, 10))
plt.imshow(conf_matrix, cmap='viridis')
plt.title('Confusion Matrix')
plt.colorbar()
plt.show()

# 输出分类报告
print('Classification Report:')
print(classification_report(y_test, mapped_predictions))

# 计算准确率
accuracy = np.sum(mapped_predictions == y_test) / len(y_test)
print(f'Clustering Accuracy: {accuracy}')

In [ ]:
def visualize_clusters_with_labels(X, y, model, perplexity=30, learning_rate=200):
    # 通过神经网络模型提取特征
    features = model.predict(X)

    # 使用t-SNE降维到二维
    tsne = TSNE(n_components=2, perplexity=perplexity, learning_rate=learning_rate, random_state=42)
    X_reduced = tsne.fit_transform(features)

    # 绘制所有聚类的散点图
    plt.figure(figsize=(12, 10))
    scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, cmap='viridis', alpha=0.6)

    # 为每个聚类的质心添加注释
    # 如果聚类数量很多，这可能会导致图表拥挤
    # unique_labels = np.unique(y)
    # for i, label in enumerate(unique_labels):
    #     xtext, ytext = np.median(X_reduced[y == label, :], axis=0)
    #     txt = f'Cluster {label}'
    #     plt.annotate(txt, (xtext, ytext), fontsize=9, ha='center')

    # 如果你想为每个点标注标签，取消注释以下代码
    for i, txt in enumerate(y):
        plt.annotate(txt, (X_reduced[i, 0], X_reduced[i, 1]), fontsize=8, alpha=0.7)

    plt.legend(handles=scatter.legend_elements()[0], labels=set(y), title="Clusters")
    plt.title('t-SNE visualization with labeled clusters')
    plt.xlabel('t-SNE feature 1')
    plt.ylabel('t-SNE feature 2')
    plt.show()


visualize_clusters_with_labels(X_test, mapped_predictions, ann_model)

In [ ]:
# from sklearn.metrics import adjusted_rand_score
# 
# adjusted_rand_index = adjusted_rand_score(y_test, predictions)
# print("调整后的兰德指数：", adjusted_rand_index)

In [ ]:
# from sklearn.metrics import normalized_mutual_info_score
# 
# normalized_mutual_info = normalized_mutual_info_score(y_test, predictions)
# print("标准化互信息：", normalized_mutual_info)